# DashBoard

## Import Libraries

In [1]:
import dash
import os
import base64

from dash.dependencies import Input,Output
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
from dash import html,Dash,dcc
import plotly.express as px

import pandas as pd
import numpy as np


## Helper Functions 

In [2]:
def split(word):
    return [char for char in word] 


def Homepoints(row,x): 
    if x==1:
        if row['FTR'] == 'H':
            return 3
        elif row['FTR'] == 'A':
            return 0
        return 1
    elif x==0:
        if row['FTR'] == 'A':
            return 3
        elif row['FTR'] == 'H':
            return 0
        return 1
    
    
def b64_image(image_filename):
    with open(image_filename, 'rb') as f:
        image = f.read()
    return 'data:image/png;base64,' + base64.b64encode(image).decode('utf-8')

## Import Data and Pre-Processing

In [3]:
#import the main Dataframe
#df has data for every single match for the season in epl
df = pd.read_csv('season-1819.csv')

#Drop redundantne columns and un necessary data 
df.drop(['Div', 'Date'], inplace = True,axis = 1)
df.drop(['B365H', 'B365D', 'B365A', 'BWH', 'BWD','BWA', 'IWH', 'IWD', 'IWA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA','VCH', 'VCD', 'VCA', 'Bb1X2', 'BbMxH', 'BbAvH', 'BbMxD', 'BbAvD','BbMxA', 'BbAvA', 'BbOU', 'BbMx>2.5', 'BbAv>2.5', 'BbMx<2.5','BbAv<2.5', 'BbAH', 'BbAHh', 'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA','PSCH', 'PSCD', 'PSCA'], inplace = True,axis = 1)

#df1 some aggregation to create a df and insights per team Home And Away games
df1 = df.groupby('AwayTeam').agg({'FTAG': 'sum'}).sort_values('FTAG',ascending= False).copy()
df1['FTHG'] = df.groupby('HomeTeam').agg({'FTHG': 'sum'})
df1['Total Goals'] = df1.FTAG + df1.FTHG

df1['Yellow Cards Home'] = df.groupby('HomeTeam').agg({'HY': 'sum'})
df1['Yellow Cards Away'] = df.groupby('AwayTeam').agg({'AY': 'sum'})
df1['Total Yellow Cards'] = df1['Yellow Cards Home'] + df1['Yellow Cards Away']

df1['Red Cards Home'] = df.groupby('HomeTeam').agg({'HR': 'sum'})
df1['Red Cards Away'] = df.groupby('AwayTeam').agg({'AR': 'sum'})
df1['Total Red Cards'] = df1['Red Cards Home'] + df1['Red Cards Away']

df1['Total Cards'] = df1['Total Yellow Cards']  + df1['Total Red Cards'] 
df1.reset_index(inplace = True)
df1.rename(columns = {'AwayTeam':'Team', 'FTHG':'Home Goals', 'FTAG':'Away Goals'}, inplace = True)



df1 = df1.merge(df[df.loc[:,'FTR'] == 'H'].groupby('HomeTeam').agg({'FTR':'count'}).fillna(0),how = 'left',left_on = 'Team',right_index = True)
df1.rename(columns = {'FTR':'Home Wins'}, inplace = True)

df1 = df1.merge(df[df.loc[:,'FTR'] == 'A'].groupby('AwayTeam').agg({'FTR':'count'}).fillna(0),how = 'left',left_on = 'Team',right_index = True)
df1.rename(columns = {'FTR':'Away Wins'}, inplace = True)


df1 = df1.merge(df[df.loc[:,'FTR'] == 'D'].groupby('HomeTeam').agg({'FTR':'count'}).fillna(0),how = 'left',left_on = 'Team',right_index = True)
df1.rename(columns = {'FTR':'Home Draw'}, inplace = True)



df1 = df1.merge(df[df.loc[:,'FTR'] == 'A'].groupby('HomeTeam').agg({'FTR':'count'}).fillna(0),how = 'left',left_on = 'Team',right_index = True)
df1.rename(columns = {'FTR':'Home Losses'}, inplace = True)

df1 = df1.merge(df[df.loc[:,'FTR'] == 'H'].groupby('AwayTeam').agg({'FTR':'count'}).fillna(0),how = 'left',left_on = 'Team',right_index = True)
df1.rename(columns = {'FTR':'Away Losses'}, inplace = True)



df1 = df1.merge(df[df.loc[:,'FTR'] == 'D'].groupby('AwayTeam').agg({'FTR':'count'}).fillna(0),how = 'left',left_on = 'Team',right_index = True)
df1.rename(columns = {'FTR':'Away Draw'}, inplace = True)



# df3.rename(columns = {'FTR':'Home Wins'}, inplace = True)
df1.fillna(0,inplace = True)

# df1

referees = df.groupby('Referee').agg({'HY':'sum','AY':'sum','HR':'sum','AR':'sum'})

referees['tota_yellow'], referees['total_red'] = referees['HY']+referees['AY'], referees['HR'] + referees['AR']
referees['no_matches'] = df.groupby('Referee').agg({'Referee':'count'})
referees['Card Booked Ratio'] = round(((referees['tota_yellow'] + referees['total_red']) / referees['no_matches']),2)
# referees
# px.sunburst(data_frame = referees,path=['no_matches',referees.index], values = 'no_matches', color = 'Card Booked Ratio')

In [5]:
df.head()


,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,Man United,Leicester,2,1,H,1,0,H,A Marriner,8,...,6,4,11,8,2,5,2,1,0,0
1,Bournemouth,Cardiff,2,0,H,1,0,H,K Friend,12,...,4,1,11,9,7,4,1,1,0,0
2,Fulham,Crystal Palace,0,2,A,0,1,A,M Dean,15,...,6,9,9,11,5,5,1,2,0,0
3,Huddersfield,Chelsea,0,3,A,0,2,A,C Kavanagh,6,...,1,4,9,8,2,5,2,1,0,0
4,Newcastle,Tottenham,1,2,A,1,2,A,M Atkinson,15,...,2,5,11,12,3,5,2,2,0,0


In [6]:
df1.head()

,Team,Away Goals,Home Goals,Total Goals,Yellow Cards Home,Yellow Cards Away,Total Yellow Cards,Red Cards Home,Red Cards Away,Total Red Cards,Total Cards,Home Wins,Away Wins,Home Draw,Home Losses,Away Losses,Away Draw
0,Man City,38,57,95,17,27,44,0,1,1,45,18,14,0.0,1.0,3,2.0
1,Liverpool,34,55,89,13,24,37,1,1,2,39,17,13,2.0,0.0,1,5.0
2,Tottenham,33,34,67,20,36,56,0,3,3,59,12,11,2.0,5.0,8,0.0
3,Crystal Palace,32,19,51,25,33,58,1,1,2,60,5,9,5.0,9.0,8,2.0
4,Man United,32,33,65,32,41,73,1,3,4,77,10,9,6.0,3.0,7,3.0


In [7]:
referees.head()

,HY,AY,HR,AR,tota_yellow,total_red,no_matches,Card Booked Ratio
Referee,,,,,,,,
A Madley,0,2,0,0,2,0,2,1.00
A Marriner,31,42,2,1,73,3,27,2.81
A Taylor,57,45,0,1,102,1,32,3.22
C Kavanagh,39,35,1,1,74,2,24,3.17
C Pawson,46,46,1,6,92,7,26,3.81


In [4]:
# df1.to_csv('Home_Away.csv')


In [8]:
#temp data frame
df_Summary = df.copy()

df_Summary['pointsHomePerGame'] = df.apply(lambda x: Homepoints(x,1), axis=1)
df_Summary['pointsAwayPerGame'] = df.apply(lambda x: Homepoints(x,0), axis=1)
df_Summary.sort_values(['HomeTeam','AwayTeam'], inplace = True)
df_Summary['Match result'] = df_Summary.FTHG.apply(str) + '-' + df_Summary.FTAG.apply(str)
df_Summary
#Heat map data after processing
df_testt = df_Summary[['HomeTeam','AwayTeam','pointsHomePerGame','Referee']].copy()

dx = pd.pivot_table(df_testt,index=['HomeTeam'], columns='AwayTeam').fillna(-1)
# dx


df_2= df_Summary.groupby('HomeTeam').agg({'pointsHomePerGame': 'sum'}).sort_values('pointsHomePerGame',ascending= False)
df_2['pointsAwayPerGame'] = df_Summary.groupby('AwayTeam').agg({'pointsAwayPerGame': 'sum'})
df_2['total_points'] = df_2['pointsAwayPerGame'] +df_2['pointsHomePerGame']
df_2.reset_index(inplace = True)

df_extra = pd.DataFrame(['1st','2nd','3rd','4th','5th','6th','7th','8th','9th','10th','11th','12th','13th','14th','15th','16th','17th','18th','19th','20th'])
df_extra1 =pd.DataFrame(['green','Blue','Blue','Blue','Orange','Orange','Orange',"Black","Black","Black","Black","Black","Black","Black","Black","Black","Black",'Red','Red','Red'])
df_extra2 = pd.DataFrame(['Win the Title & Qualified to UEFA Champions League','Qualified to UEFA Champions League','Qualified to UEFA Champions League','Qualified to UEFA Champions League','Qualified to UEFA Europa League','Qualified to UEFA Europa League','Qualified to UEFA Europa League','','','','','','','','','','','Relegation to Championship','Relegation to Championship','Relegation to Championship'])

df_2.sort_values('total_points', ascending = False,inplace = True,ignore_index = True)
df_2.reset_index( drop = True)
df_2['Pos'] = df_extra
df_2['Color'] = df_extra1
df_2['P1'] = df_extra2

#this dataframe contains BANS Values and haeder values also
# df_2

#split the same df2 based on home and away
df3 = pd.read_csv('aaa.csv',index_col= 0).iloc[0:40,:14]
df3['Comp'] = df3.Team + ' '+ df3['H/A']
df3['Total Yellow Cards'] = df1['Total Yellow Cards'].tolist()*2 
df3['Total Red Cards'] = df1['Total Red Cards'].tolist()*2 




shots = pd.read_csv('shots.csv',index_col= 0)
shots['Total Shots'] = shots.HS + shots.AS
shots['Total Shots on Target'] = shots.HST + shots.AST

avg_shots = round(sum(shots['Shots accuracy'])/shots.shape[0],2)
# df3

In [9]:
res_list = []
for i in range(21):
    res_list.append(df_Summary['Match result'][19*i:19*(i+1)].tolist())
for i in range(21):
    res_list[i].insert(i,'') 
    
POS = ['Arsenal','Bournemouth','Brighton','Burnley','Cardiff','Chelsea','Crystal Palace','Everton','Fulham','Huddersfield','Leicester','Liverpool','Man City','Man United','Newcastle','Southampton','Tottenham','Watford','West Ham','Wolves']


In [10]:
dodo_habd = html.Div(
    [
        dbc.Row(
            [
                dbc.Col(dbc.Card(id = 'goals'),width={"size": 2,"offset": 1},style={'color': 'Purple'}),
                dbc.Col(dbc.Card(id = 'Position-points'),width={"size": 2},style={'color': 'Purple'}),
                dbc.Col(dbc.Card(id = 'Total-Yellow-Cards'),width={"size": 2},style={'color': 'Purple'}),
                dbc.Col(dbc.Card(id = 'Total-Red-Cards'),width={"size": 2},style={'color': 'Purple'}),
                dbc.Col(dbc.Card(id = 'Clean-sheets'),width={"size": 2},style={'color': 'Purple'}),
            ]
        ),
    ]
)


In [11]:
H = str(round((df['FTR'].value_counts().loc['H'] /3.80),2))+'%'
D = str(round((df['FTR'].value_counts().loc['D'] /3.80),2))+'%'
A = str(round((df['FTR'].value_counts().loc['A'] /3.80),2))+'%'

SeasonCards = html.Div(
    [
        dbc.Row(
            [
                dbc.Col(dbc.Card([html.H2('Home Wins:',style={'textAlign': 'center'}),dbc.CardBody([html.H1(H, className="card-title",style={'textAlign': 'center'}),
                    ]
                ,style={'color': 'Purple'})
            ],className="card-header", color="light"
                ),width={"size": 3,"offset": 1},style={'color': 'Purple'}),
            dbc.Col(dbc.Card([html.H2('Draws:',style={'textAlign': 'center'}),dbc.CardBody([html.H1(D, className="card-title",style={'textAlign': 'center'}),
                                ]
                            ,style={'color': 'Purple'})
                        ],className="card-header", color="light"
                            ),width={"size": 4},style={'color': 'Purple'}),                
            dbc.Col(dbc.Card([html.H2('Away Wins:',style={'textAlign': 'center'}),dbc.CardBody([html.H1(A, className="card-title",style={'textAlign': 'center'}),
                    ]
                ,style={'color': 'Purple'})
            ],className="card-header", color="light"
                ),width={"size": 3},style={'color': 'Purple'}),                          
            ]
        ),
    ]
)

# print(f'Home Win {H},\nDraws {D}\nAway Win {A}')

In [ ]:

POS_default = POS[0]
image_filename = os.path.join(os.getcwd(), 'img\epl-w.png')

app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])


app.layout = html.Div(className = 'row'
                      , children=[
                                html.Div([
                                    dbc.Row(dbc.Col([html.H2(''),html.Div(html.Br(),)]))
                                    ,dbc.Row([dbc.Col(html.Div(html.Img(className = "col-4",src=b64_image(image_filename), style={'height':150, 'width':'auto'})),width={"size": 7})
                                            ,dbc.Col(html.Div(children = [html.Img(id='teamlogo',className = 'd-flex flex-column align-center', style={'height':'auto', 'width':240})] ,className = 'd-flex flex-column align-items-center'),width={"offset": 2,'size':3})])

#                                      html.Img(className = "col-4",src=b64_image(image_filename))
                                    ,html.Div([
                                    
                                            html.Div([
                                                html.Label('Club ',className='bold text-light'),
                                                dcc.Dropdown(id='club', options=[{'label': i, 'value': i} for i in POS],value = POS_default)],className='col-3'
                                                     )
                                        

#                                     ,dbc.Row(dbc.Col(html.Div(html.Img(id='teamlogo', style={'height':'auto', 'width':'30%'})),width={"offset": 7}),)

                                     ,dbc.Row(
                                            dbc.Col([html.Div(html.H2(id = 'Team-info',className= 'col text-align-center text-light'))],width={"size": 7, "offset": 1},),)


                                    ,dbc.Row(
                                            dbc.Col([html.Div(html.H2(id = 'Team-q',className= "col text-align-center text-light" ))],width={"size": 7, "offset": 1},))


                                    ,html.Br()

                                    ,dodo_habd

#                                    ,dbc.Row(
#                                         dbc.Col([html.Div([html.H5('**Green : Wining Team',className= 'text-success'),html.H5('**Blue : Champions League Qualification',className = "text-primary")
#                                             ,html.H5('**Blue : Europa League Qualification',className = "text-warning")
#                                            ,html.H5('**Black : Staying in PL',className= 'text-light'),html.H5('**Red : Relegation',className = "text-danger"),])],width={"offset": 9}))

                                             ,html.Div([html.H1('Team Summary Statistics',className= 'col text-align-center text-light mx-12 my-12')]),
                                            ],className = 'row justify-content-end')] ,style={'background-image': 'url(https://resources.premierleague.com/photos/2020/12/02/ca403c92-db70-48ac-91e2-0f2f18277133/PageHeader_RainbowLaces.png?width=1600&height=600)'}
                                        )

                        ,dbc.Row([dbc.Col(

                            html.Div([dcc.Graph(id='goals-bar')]),width={"size": 4}),
                                 dbc.Col(dcc.Graph(id='ycard-bar'),width={"size": 4}),
                                 dbc.Col(dcc.Graph(id='rcard-bar'),width={"size": 4})])     

                        ,dbc.Row([dbc.Col(

                            html.Div([dcc.Graph(id='match-bar')]),width={"size": 4}),
                                 dbc.Col(dcc.Graph(id='totshots-bar'),width={"size": 4}),
                                 dbc.Col(dcc.Graph(id='targetshots-bar'),width={"size": 4})])   
                         ,html.Div([html.Img(className = "col-4",src=b64_image(image_filename)),html.H1('Season 18-19 Summary Statistics', style = {'fontSize': 60,},className = 'text-align-center'),html.Br()], style={'textAlign': 'center', 'color':'white','background-image': 'url(https://resources.premierleague.com/photos/2020/12/02/ca403c92-db70-48ac-91e2-0f2f18277133/PageHeader_RainbowLaces.png?width=1600&height=600)','height':350})
                         
                          ,html.Div([html.H2('\nGoal Scored Per Season',style={'textAlign': 'center', 'fontSize': 35})])
                         ,dcc.Graph(figure = go.Figure(data=[go.Bar(name='Home Goals', x=df1.Team, y=df1['Home Goals'],marker_color = 'rgb(26, 83, 255)',hovertext=['Total Goals : '+s for s in df1['Total Goals'].apply(str).tolist()]),go.Bar(name='Away Goals', x=df1.Team, y=df1['Away Goals'],marker_color = 'rgb(255, 153, 0)',hovertext=['Total Goals : '+s for s in df1['Total Goals'].apply(str).tolist()])],  ).add_hline(y=df1['Total Goals'].mean()//2, line_dash="dash",
                                  annotation_text=f"Avg. Goals = {df1['Total Goals'].mean()/2}"))
                                    
                                    
#                                     px.bar(df1,x = 'Team',y = ['Home Goals', 'Away Goals'],  text_auto='.2s', title = '',hover_data=['Total Goals'],labels = {'value':'Goals','variable':'Score','Total Goals':'Total Season Goals'},color_discrete_sequence=["Green", "#54A24B"]).add_hline(y=df1['Total Goals'].mean()//2, line_dash="dash",
#                                   annotation_text=f"Avg. Goals = {df1['Total Goals'].mean()/2}"))
                        
                          ,html.Div([html.H2('Shots Attempts & Shots on Target Per Season', style={'textAlign': 'center', 'fontSize': 35})])
                          
                          ,dbc.Row([dbc.Col(

                                    html.Div([dcc.Graph(figure = go.Figure (data=[
                                            go.Bar(name='Total Shots ', x=shots.index,marker_color = 'rgb(26, 83, 255)', y=shots['HS'],hovertext=['Home Matches Shots Acc. = ' + i + '%' for i in shots['Shots accuracy Home'].apply(str).tolist()]),
                                            go.Bar(name='Tota Shots On Target', x=shots.index,marker_color = 'rgb(92, 214, 92)', y=shots['HST'],hovertext=['Home Matches Shots Acc. = ' + i + '%' for i in shots['Shots accuracy Home'].apply(str).tolist()])
                                        ]).update_layout(title='Total Shots & Shots on Target for Home Teams',yaxis_title = 'Total Home Shots per Club'))]),width={"size": 6}),
                                                                           dbc.Col(html.Div([dcc.Graph(figure = go.Figure(data=[
                                            go.Bar(name='Total Shots ', x=shots.index, y=shots['AS'],marker_color = 'rgb(255, 153, 0)',hovertext=['Away Matches Shots Acc. = ' + i + '%' for i in shots['Shots accuracy Away'].apply(str).tolist()]),
                                            go.Bar(name='Tota Shots On Target', x=shots.index, y=shots['AST'],marker_color = 'rgb(92, 214, 92)',hovertext=['Away Matches Shots Acc. = ' + i + '%' for i in shots['Shots accuracy Away'].apply(str).tolist()])
                                        ]).update_layout(title='Total Shots & Shots on Target for Away Teams',yaxis_title = 'Total Away Shots per Club'))]),width={"size": 6}),])
                          
                        ,html.Div([html.H2('Referees Booking Ratio Per Season', style={'textAlign': 'center', 'fontSize': 35})])

#                         ,dbc.Row([dbc.Col(dcc.Graph(figure =px.bar(referees, y=referees.index, x='Card Booked Ratio',  text_auto='.2s', orientation='h',color = 'no_matches', 
#                                                                                      labels={'no_matches':"Number of Matches Managed"})
# #                                                     .add_vline(x=round(sum(referees['Card Booked Ratio'])/np.count_nonzero(referees.index),2), line_dash="dash",annotation_text=f"Avg. Cards Booked By Referee = {round(sum(referees['Card Booked Ratio'])/np.count_nonzero(referees.index),2)}")
# #                                                                       .update_layout(yaxis_title="Refree",xaxis_title="Cards Booked Ratio per Match")
#                                                    )),
                                  
#                              ],)
                          ,dbc.Row([dbc.Col(dcc.Graph(figure =px.sunburst(data_frame = referees,path=['no_matches',referees.index], values = 'no_matches', color = 'Card Booked Ratio')
                                       )),
                                  
                             ],)
                          
                          
                          
                        ,html.Div([html.H2('Cards Booked Per Club for The Season', style={'textAlign': 'center', 'fontSize': 35})])

                        ,dbc.Row([dbc.Col(

                                    html.Div([dcc.Graph(figure = px.bar(df3,x = 'Team',  text_auto='.2s',y = ['Yellow Cards Home', 'Yellow Cards Away'], title = 'Yellow Cards Booked Per Season',hover_data = ['Total Yellow Cards'],labels = {'value':'Yellow Card','variable':'Booked','H/A':'Played at',},color = 'H/A',color_discrete_sequence=["#D6D013",'#FFF717']).add_hline(y=round(df1['Total Yellow Cards'].mean(),2), line_dash="dash",
                      annotation_text=f"Avg. Yellow Cards = {round(df1['Total Yellow Cards'].mean(),2)}"))
                                                ]),width={"size": 6}),
                                   dbc.Col(html.Div([dcc.Graph(figure = px.bar(df3,x = 'Team',  text_auto='.2s',y = ['Red Cards Home', 'Red Cards Away'], title = 'Red Cards Booked Per Season',hover_data = ['Total Red Cards'],labels = {'value':'Red Card','variable':'Booked','H/A':'Played at'},color = 'H/A',color_discrete_sequence=["red", "#D62728"]).add_hline(y=round(df1['Total Red Cards'].mean(),2), line_dash="dash",
                      annotation_text=f"Avg. Red Cards = {round(df1['Total Red Cards'].mean(),2)}",))
                            ]),width={"size": 6}),])
                          
                        
                         ,html.Div([html.H2('Season Matches', style={'textAlign': 'center', 'fontSize': 50})])

                         ,dbc.Row([dbc.Col(dcc.Graph(figure= go.Figure(data=go.Heatmap(
                    z=dx,
                    text=res_list,
                    x = POS,y=POS,
                    hovertext = res_list,
                    texttemplate="%{text}",
                    textfont={"size":10}, hoverinfo = ['x','y' ,'text'],xgap =2, ygap = 2
                    ,colorscale= [[0, 'Black'], [0.25, '#C40D00'], [0.5, 'Yellow'], [1, 'Green']]
                    )).update_traces(showscale=False).update_xaxes(side="top",)
                                                    .update_layout(xaxis_title="Away Teams",yaxis_title="Home Teams",font=dict(size=20,)),style={'height': '110vh'})
                                          ,width={"size": 11}),dbc.Col(html.Div([html.Img(src=b64_image('img\heat-r.png'), style={'height':'110vh'})])
                                                   ,width={"size": 1})])
                    ,html.Div([html.H2('Thank You For your Time ', style={'textAlign': 'center', 'fontSize': 22}),html.H2('Feel free to contact us for any improvements or Recommendations.\n', style={'textAlign': 'center', 'fontSize': 20})])

                                   
                      ,html.Div([html.Footer([html.P('English Premier League Analysis Project using Dash and Plotly By python For ITI'),html.P('By '),html.A("Omar Abdelaziz, ", href='https://www.linkedin.com/in/omaraaziz/',),html.A("Osama Elgendy ", href='https://www.linkedin.com/in/osamaelgendy/',)
                          
                      ] ,style={'textAlign': 'center','fontSize':20,'color' : 'white','background-image': 'url(https://resources.premierleague.com/photos/2020/12/02/ca403c92-db70-48ac-91e2-0f2f18277133/PageHeader_RainbowLaces.png?width=1600&height=600)'})
                          
                      ])
                    ])

@app.callback(
            Output(component_id='teamlogo', component_property='src'),
            
            Output(component_id='goals', component_property='children'),
            Output(component_id='Position-points', component_property='children'),
            Output(component_id='Total-Yellow-Cards', component_property='children'),
            Output(component_id='Total-Red-Cards', component_property='children'),
            Output(component_id='Clean-sheets', component_property='children'),
            
            Output(component_id='goals-bar', component_property='figure'),
            Output(component_id='ycard-bar', component_property='figure'),
            Output(component_id='rcard-bar', component_property='figure'),
            Output(component_id='match-bar', component_property='figure'),
            Output(component_id='totshots-bar', component_property='figure'),
            Output(component_id='targetshots-bar', component_property='figure'),
            
            Output(component_id='Team-info', component_property='children'),
            Output(component_id='Team-q', component_property='children'),
            
            Input(component_id='club', component_property='value'))

def goals_per_team(value):
            df_home_away = pd.read_csv('Home_Away.csv')
            df_home_away = df_home_away[df_home_away.Team==value]
            filtered_df=df1[df1.Team==value] 
            filtered_df1=df_2[df_2.HomeTeam==value] 
            df3_3 = df3[df3.Team==value] 
            sh = shots[shots.index == value]
            
            fig = px.bar(df3_3,x = 'H/A' ,y= ['Home Goals', 'Away Goals'],  text_auto='.2s',title = f'Goal Scored for {value} Per Season',labels = {'value':'Goals','variable':'Score'},color_discrete_sequence=["Green", "#54A24B"]).add_hline(y=df1['Total Goals'].mean()//2, line_dash="dash",
                      annotation_text=f"Avg. Goals = {df1['Total Goals'].mean()/2}")
            fig1 = px.bar(df3_3,x = 'H/A' ,y= ['Yellow Cards Away', 'Yellow Cards Home'],  text_auto='.2s', title = f'Yellow Card Booked for {value} Per Season',labels = {'value':'Yellow Card','variable':'Booked'},color_discrete_sequence=["Yellow",'#F6F926']).add_hline(y=df1['Total Yellow Cards'].mean()//2, line_dash="dash",
                      annotation_text=f"Avg. Yellow Cards = {df1['Total Yellow Cards'].mean()/2}")
            fig2 = px.bar(df3_3,x = 'H/A' ,y= ['Red Cards Away', 'Red Cards Home'],  text_auto='.2s', title = f'Red Card Booked for {value} Per Season',labels = {'value':'Red Card','variable':'Booked'},color_discrete_sequence=["red", "#D62728"]).add_hline(y=df1['Total Red Cards'].mean()//2, line_dash="dash",
                      annotation_text=f"Avg. Red Cards = {df1['Total Red Cards'].mean()/2}")
            fig3 = px.bar(df_home_away,x='H/A' ,y= ['Home Wins', 'Away Wins','Home Draw', 'Away Draw','Home Losses', 'Away Losses'], title = f'Matches Result for {value} Per Season',labels = {'value':'Matches Played','variable':'Matches'},color_discrete_sequence=["Green","Green", "Yellow", "Yellow", "Red", "Red"]).update_layout(showlegend=False)

            fig4 = px.bar(sh, x=['Shots accuracy Home','Shots accuracy Away'],  text_auto='.2s'+'%',y = 'H/A', title=f'Accuracy of Shots on Target for {value} Per Season',).add_vline(x=avg_shots, line_dash="dash",
                                  annotation_text=f"Avg. Shot acc. = {avg_shots}%")
            
            fig5 = px.bar(sh, x=['HS','AS'],y = 'H/A',  text_auto='.2s', title=f'Total shots for {value} in The Sesason',).add_vline(x=round(sum(shots['Total Shots'])/shots.shape[0],2), line_dash="dash",
                                  annotation_text=f"Avg. Shots per Club in the Season = {round(sum(shots['Total Shots'])/shots.shape[0],2)}")




            c1 = dbc.Card([html.H2('All Goals',style={'textAlign': 'center'}),dbc.CardBody([html.H1(filtered_df['Total Goals'], className="card-title",style={'textAlign': 'center'}),
                    ]
                ,style={'color': 'Black'})
            ],className="card-header"),
            c2 = dbc.Card([html.H2('Team Points'),dbc.CardBody([html.H1(filtered_df1['total_points'], className="card-title"),
                    ]
                ,style={'color': 'Black'}),
            ],style={'textAlign': 'center'},className="card-header"),
            c3 = dbc.Card([html.H2('Yellow Cards'),dbc.CardBody([html.H1(filtered_df['Total Yellow Cards'], className="card-title"),
                    ]
                ,style={'color': '#ECBD00'})
            ],style={'textAlign': 'center'},className="card-header"),
            c4 = dbc.Card([html.H2('Red Cards'),dbc.CardBody([html.H1(filtered_df['Total Red Cards'], className="card-title"),
                    ]
                ,style={'color': 'Red'})
            ],style={'textAlign': 'center'},className="card-header"),

            c5 = dbc.Card([html.H2('Postion'),dbc.CardBody([html.H1(filtered_df1.Pos, className="card-title"),
                    ]
                ,style={'color': filtered_df1.Color})
            ],style={'textAlign': 'center'},className="card-header")

            txt1 = f'Team : ' +filtered_df['Team']+'\n'
            txt2 = filtered_df1['P1']

            img = src=b64_image('img\\' + value + '.png')
            return (img,c1,c2,c3,c4,c5,fig,fig1,fig2,fig3,fig4,fig5,txt1,txt2)

        #     return px.bar(df1,x = 'Team',y = ['Home Goals', 'Away Goals'], title = 'Goal Scored Per Season',labels = {'value':'Goals','variable':'Score'},width=10%,height=4%)




if __name__ == "__main__":
    app.run_server()


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


In [9]:

# px.bar(df3[df3.Team == 'Arsenal'],x='H/A' ,y= ['Home Wins', 'Away Wins','Home Draw', 'Away Draw','Home Losses', 'Away Losses'],color_discrete_sequence=["Green","Green", "Yellow", "Yellow", "Red", "Red"])

go.Figure(data=go.Heatmap(
                    z=dx,
                    text=res_list,
                    x = POS,y=POS,
                    hovertext = res_list,
                    texttemplate="%{text}",
                    textfont={"size":10}, hoverinfo = ['x','y' ,'text'],xgap =2, ygap = 2
                    ,colorscale= [[0, 'Black'], [0.25, 'Red'], [0.5, 'Yellow'], [1, 'Green']]
                    )
                    ).update_traces(showscale=False)
# .update_traces(showscale=False).update_xaxes(side="top",).update_layout(title='Season Matches')
# dx

In [10]:
#some Tests 
# df1[df1.Team=='Arsenal'] 
# px.bar(df1[df1.Team=='Arsenal'] ,x = 'Team', y = ['Home Wins','Home Losses'])

# px.bar(referees, y=referees.index, x='Card Booked Ratio', orientation='h',color = 'no_matches', labels={'no_matches':"Number of Matches Managed"}).update_layout(yaxis_title="Refree",xaxis_title="Cards Booked Ratio per Match")
# px.bar(df3)
# fig = px.bar(referees, x=referees.index, y='tota_yellow')
# shots = df.groupby('HomeTeam').agg({'HS': 'sum','HST' : 'sum'})
# shots['Shots accuracy Home'] = round(shots['HST']*100/shots['HS'],2)
# shots= shots.merge(df.groupby('AwayTeam').agg({'AS': 'sum','AST': 'sum'}), right_on = 'AwayTeam',left_index=True )
# shots.rename({'AwayTeam':"Team"})
# shots['Shots accuracy Away'] = round(shots['AST']*100/shots['AS'],2)
# shots['Shots accuracy'] = round((shots['HST']+shots['AST'])*100/(shots['HS']+shots['AS']),2)
# shots['Shots accuracy Away'] + shots['Shots accuracy Home'] / 2 



px.bar(shots[shots.index == 'Arsenal'], x=['Shots accuracy Home','Shots accuracy Away'],y = 'H/A', title='Percentage of shots on target in away games',).add_vline(x=avg_shots, line_dash="dash",
                                  annotation_text=f"Avg. Shot acc. = {avg_shots}")
# # fig = px.bar(shots_home, x='on_target_percentage', y=shots_home.index, title='Percentage of shots on target in home games')
# # fig = px.bar(wide_df, facet_col="medal", color=wide_df.index)
shots

# fig2.show()

,HS,HST,Shots accuracy Home,AS,AST,Shots accuracy Away,Shots accuracy,H/A,Total Shots,Total Shots on Target
Team,,,,,,,,,,
Arsenal,256,96,37.50,0,0,0.00,36.48,Home,256,96
Bournemouth,229,82,35.81,0,0,0.00,36.47,Home,229,82
Brighton,200,53,26.50,0,0,0.00,29.59,Home,200,53
Burnley,210,66,31.43,0,0,0.00,32.03,Home,210,66
Cardiff,230,68,29.57,0,0,0.00,30.22,Home,230,68
Chelsea,325,118,36.31,0,0,0.00,32.67,Home,325,118
Crystal Palace,294,76,25.85,0,0,0.00,29.74,Home,294,76
Everton,279,85,30.47,0,0,0.00,34.14,Home,279,85
Fulham,274,86,31.39,0,0,0.00,32.89,Home,274,86


In [11]:
shots1 = shots[shots['H/A'] == 'Home']


a = ['Red','Red','Red','Red','Red','Red','Red','Red','Red','Red','Red','Red','Red','Red','Red','Red','Red','Red','Red','Red','Green','Green','Green','Green','Green','Green','Green','Green','Green','Green','Green','Green','Green','Green','Green','Green','Green','Green','Green','Green']
go.Figure(data=[
    go.Bar(name='Total Shots ', x=shots.index, y=shots['HS'],hovertext=['Home Matches Shots Acc. = ' + i + '%' for i in shots['Shots accuracy Home'].apply(str).tolist()]),
    go.Bar(name='Tota Shots On Target', x=shots.index, y=shots['HST'],hovertext=['Home Matches Shots Acc. = ' + i + '%' for i in shots['Shots accuracy Home'].apply(str).tolist()])
]).update_layout(title='Total Shots & Shots on Target for Home Teams')
# ['Total Goals : '+s for s in df1['Total Goals'].apply(str).tolist()]
# round(sum(referees['Card Booked Ratio'])/np.count_nonzero(referees.index),2)
round(df['FTR'].value_counts().loc['H'] /3.80,2)
round(df['FTR'].value_counts().loc['A'] /3.80,2)
round(df['FTR'].value_counts().loc['D'] /3.80,2)

18.68

In [13]:
# # go.Figure(data=[
# #     go.Bar(name='Total Shots ', y=shots.index, x=shots['HS'], orientation='h'),
# #     go.Bar(name='Tota Shots On Target', y=shots.index, x=shots['HST'], orientation='h')
# # ])
# # shots

# # ['Home Matches Shots Acc. = ' + i for i in shots['Shots accuracy Home'].apply(str).tolist()]
# # help(go.Bar)
# go.Figure (data=[
#     go.Bar(name='Total Shots ', x=shots.index, y=shots['HS'],hovertext=['Home Matches Shots Acc. = ' + i + '%' for i in shots['Shots accuracy Home'].apply(str).tolist()], marker_color = 'rgb(26, 83, 255)rgb(36, 143, 36)' ),
#     go.Bar(name='Tota Shots On Target', x=shots.index, y=shots['HST'],hovertext=['Home Matches Shots Acc. = ' + i + '%' for i in shots['Shots accuracy Home'].apply(str).tolist()],marker_color = 'rgb(92, 214, 92)rgb(36, 143, 36)')
# ]).update_layout(title='Total Shots & Shots on Target for Home Teams',yaxis_title = 'Total Home Shots per Club')




In [25]:
dx['Home '] = df[df.loc[:,'FTR'] == 'H'].groupby('HomeTeam').agg({'FTR':'count'}).fillna(0)

# dx = pd.concat(dx,df[df.loc[:,'FTR'] == 'A'].groupby('AwayTeam').agg({'FTR':'count'}), axis =0 )
# dx['awaywin'] =  df[df.loc[:,'FTR'] == 'A'].groupby('AwayTeam').agg({'FTR':'count'})

# dx['awayloss'] =  df[df.loc[:,'FTR'] == 'H'].groupby('AwayTeam').agg({'FTR':'count'})

# dx['homeloss'] =  df[df.loc[:,'FTR'] == 'A'].groupby('HomeTeam').agg({'FTR':'count'})

# dx['awaydraw'] = df[df.loc[:,'FTR'] == 'D'].groupby('AwayTeam').agg({'FTR':'count'})

# dx['homedraw'] =  df[df.loc[:,'FTR'] == 'D'].groupby('HomeTeam').agg({'FTR':'count'})

# dx.fillna(0)


# dx = dx.merge(df[df.loc[:,'FTR'] == 'A'].groupby('AwayTeam').agg({'FTR':'count'}),how = 'left',left_on = 'HomeTeam',right_index = True)
# df1.rename(columns = {'FTR':' Wins'}, inplace = True)
dx

,FTR_x,homewin,FTR_y,FTR
HomeTeam,,,,
Arsenal,14,14,7,7
Bournemouth,8,8,5,5
Brighton,6,6,3,3
Burnley,7,7,4,4
Cardiff,6,6,4,4
Chelsea,12,12,9,9
Crystal Palace,5,5,9,9
Everton,10,10,5,5
Fulham,6,6,1,1
